# PI7 Onderzoeksrapport
#### Dajmen Graus en Bram Verheijen 

# TODO
## Accuracy score by linear regression (werken met train en test set)
## Opdelen code stukken in kleinere stukken
## Tekst schrijven

## Data Selectie/Preperation

### Imports en inlezen data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.special import expit

df = pd.read_csv('winequality-red.csv')


### Preperatie voor Logistic Regression en Classification

In [ ]:
# Low en High quality wine set (0 for low, 1 for high), set in label 'quality_range'
bins = [0, 5, 10]

labels = [0, 1]

df_bins = df.copy()

df_bins['quality_range'] = pd.cut(x=df_bins['quality'], bins=bins, labels=labels)


## Uitwerkingen

### Multiple Linear Regression

In [ ]:
#Multiple Linear Regression
y_lin = df['quality']
X_lin = df.drop('quality',axis=1)

X_lin_train, X_lin_test, y_lin_train, y_lin_test = train_test_split(X_lin, y_lin, test_size=0.1, random_state=40)

#Sklearn solution
model_lin = LinearRegression()

model_lin.fit(X_lin_train, y_lin_train)

predictions = model_lin.predict(X_lin_test)

print("Intercept: \n", model_lin.intercept_)
print("Coefficients: \n", model_lin.coef_)


In [ ]:
#SM (statsmodel.api) solution
X_lin_train = sm.add_constant(X_lin_train)

model_sm = sm.OLS(y_lin_train, X_lin_train).fit()
predictions_sm = model_sm.predict(X_lin_train)

print_model = model_sm.summary()
print(print_model)

In [ ]:
#Accuracy Testing
train_accuracy = model_lin.score(X_lin_train, y_lin_train)
test_accuracy = model_lin.score(X_lin_test, y_lin_test)

y_lin_prediction = model_lin.predict(X_lin_test)

lin_score = r2_score(y_lin_test, y_lin_prediction)

print('One-vs-rest', '-'*35, 
      'Accuracy in Train Group   : {:.2f}'.format(train_accuracy), 
      'Accuracy in Test  Group   : {:.2f}'.format(test_accuracy), sep='\n')

print("\n")

print('R2 and Mean errors', '-'*35, 
      'R2-score   : {:.2f}'.format(lin_score), 
      'Mean Squared Error   : {:.2f}'.format(mean_squared_error(y_lin_test,y_lin_prediction)),
      'Root Mean Squared Error   : {:.2f}'.format(np.sqrt(mean_squared_error(y_lin_test,y_lin_prediction))), sep='\n')


### Logistic Regression

In [ ]:

#Set X and y, also split into train and test sets
X_log = df_bins[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']]

y_log = df_bins['quality_range']

X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X_log, y_log, test_size=0.1, random_state=40)

In [ ]:
#Logistic regression
model_lr = LogisticRegression(random_state=40, max_iter=1600)

model_lr.fit(X_log_train, y_log_train)

train_accuracy = model_lr.score(X_log_train, y_log_train)
test_accuracy = model_lr.score(X_log_test, y_log_test)

print('One-vs-rest', '-'*35, 
      'Accuracy in Train Group   : {:.2f}'.format(train_accuracy), 
      'Accuracy in Test  Group   : {:.2f}'.format(test_accuracy), sep='\n')


In [ ]:
#Plot Logistic Regression
plt.figure(1, figsize=(4, 3))
plt.clf()
plt.scatter(X_log_train['fixed acidity'], y_log_train, label="test data", color='black')

log_coef = model_lr.coef_

log_coef = log_coef.reshape(11,1)

loss = expit(X_log_test['fixed acidity'] * log_coef[0] + model_lr.intercept_).ravel()
plt.plot(X_log_test, loss, label="Logistic Regression model fixed acidity", color="red")
print(loss)

plt.ylabel("Good/Bad Wine Quality")
plt.xlabel("fixed acidity")
plt.xlim(2,18)


In [ ]:

log_predictions = model_lr.predict(X_log_test)

#confusion matrix
score = round(accuracy_score(y_log_test, log_predictions), 3)
confusionMatrix_test = cm(y_log_test, log_predictions)
sns.heatmap(confusionMatrix_test, annot=True, fmt='.0f')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

#Array confusion matrix
log_pred_test = model_lr.predict(X_log_test)
log_pred_train = model_lr.predict(X_log_train)

cm = cm(y_log_test, log_pred_test)
print(cm)

In [ ]:
#ROC-curve

probs = model_lr.predict_proba(X_log_test)[:,1] #predifct probabilities for the test data

fpr, tpr, thresholds =  roc_curve(y_log_test, probs) #Get ROC Curve

plt.figure(figsize=(8,5))
#Plot ROC curve
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('1 - Specificity Score')
plt.ylabel('Recall Score')
plt.title('ROC-Curve')
plt.show()


## Conclusie